In [ ]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import geemap, json
from city_metrix.layers import Layer
from city_metrix.layers.layer import get_image_collection
import ee
# ee.Authenticate()
ee.Initialize()

FILEPATH = 'C:/Users/theodore.wong/danida_africa'

In [ ]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

In [ ]:
class Hand(Layer):
    def __init__(self, spatial_resolution=30, **kwargs):
        super().__init__(**kwargs)
        self.spatial_resolution = spatial_resolution

    def get_data(self, bbox):
        dataset = ee.ImageCollection(ee.Image("users/gena/GlobalHAND/30m/hand-1000").focal_mean(0.1))
        hand = ee.ImageCollection(dataset
             .filterBounds(ee.Geometry.BBox(*bbox))
             .select('b1')
             .sum()
             )

        data = get_image_collection(hand, bbox, self.spatial_resolution, "hand")
        return data.b1

In [ ]:
for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] == 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
            feature = geemap.geojson_to_ee(json_obj)
            geom = geemap.ee_to_gdf(feature)['geometry'][0]
            handmap = Hand().get_data(geom.bounds).rename("HAND_lt_1m")
            handmap.rio.to_raster('{0}/HANDlt1m-{1}.tif'.format(FILEPATH, cityname.replace(' ', '-')))
            print(cityname)